In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from utils import chart_price, plot, perf, v_backtester, c_backtester, perf_var, summary
import sys
sys.path.append('/home/tomek/ib_tools')
import matplotlib.pyplot as plt
%matplotlib inline
from tester import run, simulate, calibrate

In [2]:
def ema(data, periods, smooth, vol):
    data = data.copy()
    data['ema_fast'] = data.close.ewm(span=periods).mean()
    data['ema_slow'] = data.close.ewm(span=periods*3).mean()
    data['ema_diff'] = data.ema_fast - data.ema_slow
    data['ema_diff_norm'] = data.ema_diff / vol #data.ema_diff.ewm(span=vol_lookback).std()
    #data['ema_diff_norm'] = data.ema_diff / data.ema_diff.rolling(periods).std()
    #data[f'ema_{periods}'] = data['ema_diff_norm'].ewm(span=smooth).mean()
    data[f'ema_{periods}'] = data['ema_diff_norm']
    return data[f'ema_{periods}']

In [3]:
def roc(data, periods, smooth, vol):
    data = data.copy()
    data['roc'] = data.close.pct_change(periods) / vol
    data[f'roc_{periods}'] = smooth(data['roc'], periods/3)
    return data[f'roc_{periods}']

In [4]:
data = run('NQ', roc)

In [5]:
b = v_backtester(data.open, data.forecast, 7)
r = perf_var(b, False, slippage=1.5)

estimated min-tick: 0.25


In [6]:
r.df

,price,indicator,signal,position,transaction,slippage,curr_price,base_price,pnl
date,,,,,,,,,
2019-01-02 02:23:00,6354.25,14.754206,1,0,0,0.0000,0.0,0.0,0.0000
2019-01-02 03:38:00,6377.50,5.668928,0,1,1,0.1875,0.0,0.0,-0.1875
2019-01-02 05:41:00,6296.00,3.407423,0,0,-1,0.1875,6296.0,6377.5,-81.6875
2019-01-02 08:27:00,6278.75,-0.776656,0,0,0,0.0000,0.0,0.0,0.0000
2019-01-02 09:21:00,6257.25,-4.245627,0,0,0,0.0000,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...
2019-09-04 16:59:00,7679.25,1.044707,0,0,0,0.0000,0.0,0.0,0.0000
2019-09-04 17:24:00,7682.00,3.196337,0,0,0,0.0000,0.0,0.0,0.0000
2019-09-04 17:41:00,7686.00,6.405465,0,0,0,0.0000,0.0,0.0,0.0000


In [7]:
r.positions

,date_o,open,date_c,close,pnl,duration
0,2019-01-02 03:38:00,6377.50,2019-01-02 05:41:00,-6296.00,-81.875,02:03:00
1,2019-01-02 11:59:00,-6191.50,2019-01-02 14:17:00,6216.75,-25.625,02:18:00
2,2019-01-02 16:18:00,6298.25,2019-01-02 16:43:00,-6282.75,-15.875,00:25:00
3,2019-01-02 16:53:00,6315.25,2019-01-02 18:09:00,-6322.00,6.375,01:16:00
4,2019-01-02 18:57:00,6364.50,2019-01-02 21:18:00,-6354.00,-10.875,02:21:00
...,...,...,...,...,...,...
356,2019-08-30 10:00:00,7718.75,2019-08-30 16:24:00,-7724.00,4.875,06:24:00
357,2019-09-02 15:22:00,7687.00,2019-09-02 16:19:00,-7663.00,-24.375,00:57:00
358,2019-09-03 20:50:00,-7591.25,2019-09-03 21:15:00,7601.00,-10.125,00:25:00
359,2019-09-04 08:49:00,7685.00,2019-09-04 15:29:00,-7698.25,12.875,06:40:00
